In [2]:
!pip install -q transformers datasets peft scikit-learn pandas pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.5 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigfram

In [3]:
from datasets import Dataset
import torch
from transformers import (
    BlipForQuestionAnswering,
    BlipProcessor,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from PIL import Image

# 1. Dataset Preparation -----------------------------------------------
# Load and flatten your dataset
df = pd.read_csv("/kaggle/input/merged-one/merged_vqa_dataset_output.csv")
BASE_IMAGE_DIR = "/kaggle/input/abo-dataset/images/small"

# Flatten into (image_path, question, answer) format
samples = []
for _, row in df.iterrows():
    for q_col, a_col in zip(['q1', 'q2', 'q3'], ['a1', 'a2', 'a3']):
        samples.append({
            "image_path": os.path.join(BASE_IMAGE_DIR, row['path']),
            "question": row[q_col],
            "answer": row[a_col]
        })

# Split into train/test
train_df, test_df = train_test_split(pd.DataFrame(samples), test_size=0.2, random_state=42)


2025-05-16 15:26:31.132141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747409191.331295      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747409191.391854      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
test_df

,image_path,question,answer
9743,/kaggle/input/abo-dataset/images/small/28/2881...,What color is it?,Blue
31595,/kaggle/input/abo-dataset/images/small/30/3020...,What is the frame made of?,Wood
28281,/kaggle/input/abo-dataset/images/small/52/52e2...,What is in the image?,Handle
42139,/kaggle/input/abo-dataset/images/small/56/5678...,What color is it?,Gray
33152,/kaggle/input/abo-dataset/images/small/0f/0fd6...,How many screws are there?,Two
...,...,...,...
4523,/kaggle/input/abo-dataset/images/small/c0/c094...,What is the container made of?,Plastic
7231,/kaggle/input/abo-dataset/images/small/07/07c2...,What is the height?,6.0in
21016,/kaggle/input/abo-dataset/images/small/50/5019...,What color is the background?,blue
9242,/kaggle/input/abo-dataset/images/small/a2/a2bb...,How much is the height?,12in


In [5]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
test_dataset = Dataset.from_pandas(test_df, preserve_index=False)


In [6]:
wanted_cols = {"image_path", "question", "answer"}
for ds in [train_dataset, test_dataset]:
    extra_cols = [col for col in ds.column_names if col not in wanted_cols]
    if extra_cols:
        ds = ds.remove_columns(extra_cols)
wanted_cols = {"image_path", "question", "answer"}
for ds in [train_dataset, test_dataset]:
    extra_cols = [col for col in ds.column_names if col not in wanted_cols]
    if extra_cols:
        ds = ds.remove_columns(extra_cols)


In [7]:
from PIL import Image
from transformers import BlipProcessor

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

def transform(batch):
    images = [Image.open(path).convert("RGB") for path in batch["image_path"]]
    encoding = processor(
        images,
        batch["question"],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=32
    )
    labels = processor.tokenizer(
        batch["answer"],
        padding="max_length",
        truncation=True,
        max_length=32,
        return_tensors="pt"
    ).input_ids

    batch["pixel_values"] = encoding["pixel_values"]
    batch["input_ids"] = encoding["input_ids"]
    batch["attention_mask"] = encoding["attention_mask"]
    batch["labels"] = labels
    return batch


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [8]:
train_dataset.set_transform(transform)
test_dataset.set_transform(transform)
train_dataset

Dataset({
    features: ['image_path', 'question', 'answer'],
    num_rows: 36883
})

In [9]:
from transformers import BlipForQuestionAnswering
from peft import LoraConfig, get_peft_model

model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["query", "key", "value", "fc1", "fc2"],  # Adjust if needed after inspecting model.named_modules()
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

trainable params: 2,506,752 || all params: 387,179,324 || trainable%: 0.6474


In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./blip-lora-vqa",
    per_device_train_batch_size=4,   # Lower if you have OOM errors
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    fp16=True,
    learning_rate=1e-4,
    logging_steps=50,
    remove_unused_columns=False,
    report_to="none"
)


In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    # tokenizer=processor,  # Optional, warning may appear but can be ignored for now
)


No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [12]:
train_results = trainer.train()
trainer.save_model("./blip-lora-vqa-final")
eval_results = trainer.evaluate()
print(f"Final evaluation results: {eval_results}")


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
50,9.602000
100,8.758200
150,8.454200
200,8.330700
250,8.272700
300,8.255500
350,8.227600
400,8.236400
450,8.223900
500,8.210500


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

Final evaluation results: {'eval_runtime': 666.1398, 'eval_samples_per_second': 13.842, 'eval_steps_per_second': 1.731, 'epoch': 3.0}


In [15]:
import os

model_dir = "./blip-lora-vqa-final"
print("Directory exists:", os.path.exists(model_dir))
print("Contents:", os.listdir(model_dir) if os.path.exists(model_dir) else "N/A")


Directory exists: True
Contents: ['README.md', 'training_args.bin', 'adapter_model.safetensors', 'adapter_config.json']


In [16]:
!zip -r -s 300m /kaggle/working/blip-lora-vqa-split.zip /kaggle/working/blip-lora-vqa


  adding: kaggle/working/blip-lora-vqa/ (stored 0%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-10000/ (stored 0%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-10000/scaler.pt (deflated 60%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-10000/rng_state.pth (deflated 25%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-10000/README.md (deflated 66%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-10000/training_args.bin (deflated 52%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-10000/trainer_state.json (deflated 79%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-10000/adapter_model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 7%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-10000/optimizer.pt (deflated 8%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-10000/scheduler.pt (deflated 55%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-10000/adapter_config.json (deflated 54%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-6000/ (stored 0%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-6000/scaler.pt (deflated 60%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-6000/rng_state.pth (deflated 25%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-6000/README.md (deflated 66%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-6000/training_args.bin (deflated 52%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-6000/trainer_state.json (deflated 78%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-6000/adapter_model.safetensors (deflated 7%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-6000/optimizer.pt (deflated 8%)
  adding: kaggle/working/blip-lora-vqa/checkpoint-6000/schedu

In [20]:
!zip -r /kaggle/working/blip-lora-vqa-final.zip /kaggle/working/blip-lora-vqa-final


  adding: kaggle/working/blip-lora-vqa-final/ (stored 0%)
  adding: kaggle/working/blip-lora-vqa-final/README.md (deflated 66%)
  adding: kaggle/working/blip-lora-vqa-final/training_args.bin (deflated 52%)
  adding: kaggle/working/blip-lora-vqa-final/adapter_model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 7%)
  adding: kaggle/working/blip-lora-vqa-final/adapter_config.json (deflated 54%)


In [17]:
import torch
from transformers import BlipForQuestionAnswering, BlipProcessor
from PIL import Image

# Load model and processor
model_1 = BlipForQuestionAnswering.from_pretrained("./blip-lora-vqa-final")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_1 = model_1.to(device)
model_1.eval()

# Preview predictions for the first 10 rows
for idx, row in test_df.head(10).iterrows():
    image = Image.open(row['image_path']).convert('RGB')
    question = row['question']
    true_answer = row['answer']

    # Preprocess
    inputs = processor(image, question, return_tensors="pt").to(device)

    # Generate prediction
    with torch.no_grad():
        out = model_1.generate(**inputs)
    predicted_answer = processor.decode(out[0], skip_special_tokens=True)

    print(f"Row {idx}:")
    print(f"  Question: {question}")
    print(f"  True Answer: {true_answer}")
    print(f"  Predicted Answer: {predicted_answer}")
    print("-" * 50)


Row 9743:
  Question: What color is it?
  True Answer: Blue
  Predicted Answer: blue
--------------------------------------------------
Row 31595:
  Question: What is the frame made of?
  True Answer: Wood
  Predicted Answer: wood
--------------------------------------------------
Row 28281:
  Question: What is in the image?
  True Answer: Handle
  Predicted Answer: handle
--------------------------------------------------
Row 42139:
  Question: What color is it?
  True Answer: Gray
  Predicted Answer: grey
--------------------------------------------------
Row 33152:
  Question: How many screws are there?
  True Answer: Two
  Predicted Answer: two
--------------------------------------------------
Row 28154:
  Question: What is the height?
  True Answer: 96cm
  Predicted Answer: 85cm
--------------------------------------------------
Row 39757:
  Question: What color is the case?
  True Answer: Pink
  Predicted Answer: pink
--------------------------------------------------
Row 19645:

In [21]:
import time
import torch
from transformers import BlipForQuestionAnswering, BlipProcessor
from PIL import Image

# Start timing
start_time = time.time()

# 1. Load your fine-tuned model and processor
model_1 = BlipForQuestionAnswering.from_pretrained("./blip-lora-vqa-final")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

# 2. Move model to device (CPU or CUDA)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_1 = model_1.to(device)
model_1.eval()

# 3. Loop over your test DataFrame and generate answers
pred_answers = []

for idx, row in test_df.iterrows():
    image = Image.open(row['image_path']).convert('RGB')
    question = row['question']

    # Preprocess inputs
    inputs = processor(image, question, return_tensors="pt").to(device)

    # Generate answer
    with torch.no_grad():
        out = model_1.generate(**inputs)
    answer = processor.decode(out[0], skip_special_tokens=True)
    pred_answers.append(answer)

# 4. Add predictions to your DataFrame as a new column
test_df['predicted_answer'] = pred_answers

# End timing
end_time = time.time()
elapsed_time = end_time - start_time

# 5. Display or save results
print(test_df[['image_path', 'question', 'answer', 'predicted_answer']].head())
print(f"\n⏱️ Total time taken: {elapsed_time:.2f} seconds")


                                              image_path  \
9743   /kaggle/input/abo-dataset/images/small/28/2881...   
31595  /kaggle/input/abo-dataset/images/small/30/3020...   
28281  /kaggle/input/abo-dataset/images/small/52/52e2...   
42139  /kaggle/input/abo-dataset/images/small/56/5678...   
33152  /kaggle/input/abo-dataset/images/small/0f/0fd6...   

                         question  answer predicted_answer  
9743            What color is it?    Blue             blue  
31595  What is the frame made of?    Wood             wood  
28281       What is in the image?  Handle           handle  
42139           What color is it?    Gray             grey  
33152  How many screws are there?     Two              two  

⏱️ Total time taken: 1096.37 seconds


In [22]:
# Assuming 'test_df' already has columns: 'answer' (ground truth) and 'predicted_answer'
# We'll add 'is_true' column: 1 if prediction matches ground truth, else 0

# Normalize answers for comparison (optional but recommended)
def normalize(text):
    return str(text).strip().lower()

test_df['is_true'] = (test_df['predicted_answer'].apply(normalize) == test_df['answer'].apply(normalize)).astype(int)

# Count number of true and false predictions
num_true = test_df['is_true'].sum()
num_false = len(test_df) - num_true

print(f"Number of correct predictions (is_true=1): {num_true}")
print(f"Number of incorrect predictions (is_true=0): {num_false}")

# Save to CSV
test_df.to_csv('LORS_PRED_8.csv', index=False)
print("Results saved to LORS_PRED_8.csv")


Number of correct predictions (is_true=1): 6066
Number of incorrect predictions (is_true=0): 3155
Results saved to LORS_PRED_8.csv


In [24]:
def normalize_answer(s):
    """Normalize answer for more accurate comparison."""
    s = re.sub(r'\b(a|an|the)\b', ' ', s.lower())
    s = re.sub(r'[^\w\s]', '', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def calculate_bertscore(pred, ref, tokenizer, model, device):
    """Calculate BERTScore between prediction and reference."""
    pred_tokens = tokenizer(pred, return_tensors='pt', padding=True, truncation=True).to(device)
    ref_tokens = tokenizer(ref, return_tensors='pt', padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        pred_outputs = model(**pred_tokens)
        ref_outputs = model(**ref_tokens)
    
    pred_embedding = pred_outputs.last_hidden_state[:, 0, :]
    ref_embedding = ref_outputs.last_hidden_state[:, 0, :]
    
    pred_embedding = pred_embedding / pred_embedding.norm(dim=1, keepdim=True)
    ref_embedding = ref_embedding / ref_embedding.norm(dim=1, keepdim=True)
    
    similarity = torch.matmul(pred_embedding, ref_embedding.transpose(0, 1)).item()
    return similarity

def exact_match(pred, ref):
    """Check if prediction exactly matches reference after normalization."""
    return normalize_answer(pred) == normalize_answer(ref)

def token_match(pred, ref):
    """Check if the tokens in prediction match the reference."""
    pred_tokens = normalize_answer(pred).split()
    ref_tokens = normalize_answer(ref).split()
    return Counter(pred_tokens) == Counter(ref_tokens)

In [26]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load pretrained BERT model and tokenizer
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased").to(device)
bert_model.eval()

# Compute BERTScore for each prediction-reference pair
bert_scores = []
for pred, ref in zip(test_df['predicted_answer'], test_df['answer']):
    pred = str(pred)
    ref = str(ref)
    score = calculate_bertscore(pred, ref, bert_tokenizer, bert_model, device)
    bert_scores.append(score)

# Add BERTScore column to DataFrame
test_df['bertscore'] = bert_scores

# Print average BERTScore
print(f"📊 Average BERTScore: {sum(bert_scores) / len(bert_scores):.4f}")

# (Optional) Save the results to CSV
test_df.to_csv("LORA_PRED_8_with_bertscore.csv", index=False)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

📊 Average BERTScore: 0.9751


In [28]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel

# Load CSV file
df = pd.read_csv("/kaggle/input/lora-pred-16/LORS_PRED_16.csv")

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load BERT tokenizer and model
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased").to(device)
bert_model.eval()

# BERTScore function (same as yours)
def calculate_bertscore(pred, ref, tokenizer, model, device):
    pred_tokens = tokenizer(pred, return_tensors='pt', padding=True, truncation=True).to(device)
    ref_tokens = tokenizer(ref, return_tensors='pt', padding=True, truncation=True).to(device)

    with torch.no_grad():
        pred_outputs = model(**pred_tokens)
        ref_outputs = model(**ref_tokens)

    pred_embedding = pred_outputs.last_hidden_state[:, 0, :]
    ref_embedding = ref_outputs.last_hidden_state[:, 0, :]

    pred_embedding = pred_embedding / pred_embedding.norm(dim=1, keepdim=True)
    ref_embedding = ref_embedding / ref_embedding.norm(dim=1, keepdim=True)

    similarity = torch.matmul(pred_embedding, ref_embedding.transpose(0, 1)).item()
    return similarity

# Compute BERTScores
bert_scores = []
for pred, ref in zip(df['predicted_answer'], df['answer']):
    pred = str(pred)
    ref = str(ref)
    score = calculate_bertscore(pred, ref, bert_tokenizer, bert_model, device)
    bert_scores.append(score)

# Add to DataFrame
df['bertscore'] = bert_scores

# Print average BERTScore
print(f"📊 Average BERTScore: {sum(bert_scores) / len(bert_scores):.4f}")

# Save result (optional)
df.to_csv("LORA_PRED_16_with_bertscore.csv", index=False)


📊 Average BERTScore: 0.9757
